In [1]:
from library.utils import PauliToQuditOperator
import numpy as np

In [10]:
input_str = "ZXXY"

op = PauliToQuditOperator(input_str, qudit_dim_size=5)

test = op @ op.conj().T
print(op.shape)
print(test.shape)
test = np.array(test)
id_matrix = np.eye(len(op), dtype=np.complex64)

vector_size = len(op)
num_qubits = len(input_str)
num_levels = vector_size ** (1/num_qubits)
print(int(num_levels))

print(np.all(test==id_matrix))

(625, 625)
(625, 625)
5
True
